# Cisco DevNet - RestAPI

## 什么是 API ？


![API](assert/api.png)

## 0. 模拟一个API交互

In [4]:
import time

class SERVER():
    def __init__(self):
        self.url = 'yeslab.com'
        self.data = {'route':['to_192.168.0.0', 'to_172.18.0.0']}
        self.keys = list(self.data.keys())
        self.announce()
        
    def announce(self):
        print("我的地址是：{}".format(self.url))
        print("我能提供的服务是：{}".format(self.keys[0]))
        
    def response(self, receive):
        msg = receive()
        print("【SERVER】API请求处理中。。。")
        time.sleep(2)
        if msg == self.url + '/' + self.keys[0]:
            return self.data['route']
        
server = SERVER()

我的地址是：yeslab.com
我能提供的服务是：route


In [5]:
@server.response
def client_send():
    print("【CLIENT】API请求发送中。。。")
    time.sleep(2)
    return "yeslab.com/route"

print(client_send)

【CLIENT】API请求发送中。。。
【SERVER】API请求处理中。。。
['to_192.168.0.0', 'to_172.18.0.0']


![API2](assert/api2.png)

* REST is "just another use for the HTTP Protocol." A REST API service sends all requests and responses over TCP/TLS connections using text.
* REST is an API framework built on HTTP, and the interface points are often for web services. When you combine REST and API, you see a simple service interface that enables applications or people to use the HTTP protocol to request objects or information.

![HTTP](assert/http.png)

### HTTP 请求方法
* POST 新建
* GET 读取
* PUT 更新
* PATCH 更新
* DELETE 删除

![restapi](assert/restapi.png)

![url](assert/url.png)

## APIC-EM APIs
https://github.com/CiscoDevNet/apicem-apis-with-python-sample-codes

### 1. 输入远端设备信息

In [6]:
import requests
import json
from helper import apicem

requests.packages.urllib3.disable_warnings()

apicem_ip = "sandboxapicem.cisco.com"
username = "devnetuser"
password = "Cisco123!"
version = "v1"

### 2. 拿到访问授权

In [8]:
r_json = {
  "username": username,
  "password": password
}

post_url = "https://" + apicem_ip + "/api/" + version + "/ticket"

headers = {'content-type':'application/json'}

resp = requests.post(post_url, json.dumps(r_json), headers=headers, verify=False)

In [11]:
resp.status_code

200

In [12]:
resp.json()

{'response': {'serviceTicket': 'ST-32823-zg5wjHm2LR1xvVqY5CFI-cas',
  'idleTimeout': 1800,
  'sessionTimeout': 21600},
 'version': '1.0'}

### 3. GET 查看用户信息

In [13]:
user_resp = apicem.get(apicem_ip, version, username, password, api='user')


Executing GET 'https://sandboxapicem.cisco.com/api/v1/user'

GET 'user' Status:  200 



In [23]:
user_resp.json()

{'response': [{'username': 'devnetuser',
   'authorization': [{'scope': 'ALL', 'role': 'ROLE_POLICY_ADMIN'}],
   'authSource': 'internal'}],
 'version': '1.0'}

### 3.1 GET 角色列表失败

In [24]:
role_resp = apicem.get(apicem_ip, version, username, password, api='user/role')
role_resp.json()


Executing GET 'https://sandboxapicem.cisco.com/api/v1/user/role'

GET 'user/role' Status:  403 



{'response': {'errorCode': 'RBAC',
  'message': 'You do not have permissions to access this resource',
  'detail': 'You do not have permissions to access this resource'},
 'version': '1.0'}

### 4. POST 添加用户失败

In [26]:
add_user_data = {
  "username": "yeslab123",
  "password": "yeslab456!",
  "authSource": "internal",
  "authorization": [
    {
      "scope": "ALL",
      "role": "ROLE_POLICY_ADMIN"
    }
  ],
  "oldPassword": "yeslab456!"
}

post_user_resp = apicem.post(apicem_ip, version, username, password, api='user', data=add_user_data)


Executing POST 'https://sandboxapicem.cisco.com/api/v1/user'

POST 'user' Status:  403 



In [28]:
post_user_resp.json()

{'response': {'errorCode': 'RBAC',
  'message': 'You do not have permissions to access this resource',
  'detail': 'You do not have permissions to access this resource'},
 'version': '1.0'}

### 5. POST 添加策略标签成功

In [35]:
policy_tag = {
  "policyTag": "yeslab_hello"
}

post_tag_resp = apicem.post(apicem_ip, version, username, password, api='policy/tag', data=policy_tag)


Executing POST 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

POST 'policy/tag' Status:  202 



In [36]:
look_tag_resp = apicem.get(apicem_ip, version, username, password, api='policy/tag')
look_tag_resp.json()


Executing GET 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

GET 'policy/tag' Status:  200 



{'response': [{'policyTag': 'YESLAB'},
  {'policyTag': 'luisga'},
  {'policyTag': 'yeslab_hello'},
  {'policyTag': 'Vijoy_Added_this'},
  {'policyTag': 'Student_Added_This'},
  {'policyTag': 'lala'},
  {'policyTag': 'UTCN'},
  {'policyTag': 'Bryan_gonz'},
  {'policyTag': 'vanakkad'},
  {'policyTag': 'mahovich'},
  {'policyTag': 'GMR'},
  {'policyTag': 'RemoteSite'},
  {'policyTag': 'lpj'},
  {'policyTag': 'devent_tag'},
  {'policyTag': 'RemoteSite1'},
  {'policyTag': 'duno'},
  {'policyTag': 'test'},
  {'policyTag': 'TheBigNetwork'},
  {'policyTag': 'new_policy_tag'}],
 'version': '1.0'}

### 6. DELETE 删除标签

In [39]:
policy_tag = {
  "policyTag": "GMR"
}

delete_tag_resp = apicem.delete(apicem_ip, version, username,password, api='policy/tag', params=policy_tag)

look_tag_resp = apicem.get(apicem_ip, version, username, password, api='policy/tag')
look_tag_resp.json()


Executing DELETE 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

DELETE 'policy/tag' Status:  202 


Executing GET 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

GET 'policy/tag' Status:  200 



{'response': [{'policyTag': 'YESLAB'},
  {'policyTag': 'luisga'},
  {'policyTag': 'Vijoy_Added_this'},
  {'policyTag': 'Student_Added_This'},
  {'policyTag': 'lala'},
  {'policyTag': 'UTCN'},
  {'policyTag': 'Bryan_gonz'},
  {'policyTag': 'vanakkad'},
  {'policyTag': 'mahovich'},
  {'policyTag': 'RemoteSite'},
  {'policyTag': 'lpj'},
  {'policyTag': 'devent_tag'},
  {'policyTag': 'RemoteSite1'},
  {'policyTag': 'duno'},
  {'policyTag': 'test'},
  {'policyTag': 'TheBigNetwork'},
  {'policyTag': 'new_policy_tag'}],
 'version': '1.0'}